In [ ]:
# 1. Ultralytics ve Gerekli Kütüphaneleri Kur
!pip install ultralytics -q
from ultralytics import YOLO
import os
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

# 2. Kaggle Kimlik Doğrulama
from google.colab import files
if not os.path.exists("/root/.kaggle/kaggle.json"):
    print("Lütfen kaggle.json dosyanızı yükleyin:")
    uploaded = files.upload()
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

# 3. Veri Setini İndir ve Çıkar
!kaggle datasets download -d andrewmvd/face-mask-detection
!unzip -q face-mask-detection.zip -d dataset

In [ ]:
# Sınıfları tanımlayalım
classes = ["with_mask", "without_mask", "mask_weared_incorrect"]

def convert_voc_to_yolo(xml_path, output_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    with open(output_path, 'w') as f:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in classes: continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            
            # YOLO formatına dönüştürme (merkez_x, merkez_y, genişlik, yükseklik) normalize edilmiş
            bb = ((b[0] + b[1]) / 2.0 / w, (b[2] + b[3]) / 2.0 / h,
                  (b[1] - b[0]) / w, (b[3] - b[2]) / h)
            f.write(f"{cls_id} {' '.join([f'{a:.6f}' for a in bb])}\n")

# Klasörleri oluştur
os.makedirs('yolo_data/images/train', exist_ok=True)
os.makedirs('yolo_data/images/val', exist_ok=True)
os.makedirs('yolo_data/labels/train', exist_ok=True)
os.makedirs('yolo_data/labels/val', exist_ok=True)

# Verileri parçala (Train/Val split)
all_images = sorted(glob('dataset/images/*.png'))
train_imgs, val_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

def move_files(img_list, split):
    for img_path in img_list:
        name = os.path.basename(img_path).split('.')[0]
        # Resimleri taşı
        shutil.copy(img_path, f'yolo_data/images/{split}/{name}.png')
        # XML'i çevir ve kaydet
        convert_voc_to_yolo(f'dataset/annotations/{name}.xml', f'yolo_data/labels/{split}/{name}.txt')

move_files(train_imgs, 'train')
move_files(val_imgs, 'val')

# data.yaml dosyasını oluştur (YOLO'nun eğitimi için gerekli)
import yaml
config = {
    'path': '/content/yolo_data',
    'train': 'images/train',
    'val': 'images/val',
    'names': {0: 'Maskeli', 1: 'Maskesiz', 2: 'Hatalı Maske'}
}
with open('data.yaml', 'w') as f:
    yaml.dump(config, f)

In [ ]:
# Modeli Yükle (Nano model hızlıdır, Medium veya Large A100'de daha iyi sonuç verir)
model = YOLO('yolov8m.pt') # m (medium) versiyonu dengelidir

# Eğitimi başlat
results = model.train(
    data='data.yaml',
    epochs=100,           # 100'e çıkardık
    imgsz=640,
    batch=32,             # A100 için 64 bile yapabilirsin ama 32 de güvenli
    patience=20,          # Eğer 20 epoch boyunca iyileşme olmazsa durur (Early Stopping)
    optimizer='AdamW',     # Küçük veri setlerinde ve dengesiz sınıflarda daha kararlı olabilir
    lr0=0.001,            # Öğrenme hızını biraz daha hassas başlatalım
    augment=True,         # Veri artırmayı kesinlikle açık tut (zaten varsayılan açık)
    project='mask_project',
    name='v2_100_epochs'
)

In [ ]:
from google.colab import files
# En iyi modeli indir
files.download('mask_project/v1/weights/best.pt')